# Model Distillation for Function Calling

This notebook is part of a series demonstrating advanced model distillation techniques for creating specialized, function-calling-aware models. The goal is to distill the knowledge from a large language model (Amazon Nova Premier) into a smaller, more efficient model while maintaining high-quality function calling capabilities.

## Learning Objectives
- Prepare training data for function calling model distillation
- Design structured output formats for consistent function parameter generation
- Implement function selection and parameter extraction
- Create evaluation datasets for measuring function calling accuracy

## Dataset: Berkeley Function Calling Leaderboard (BFCL) V2 Live
We use the Berkeley Function Calling Leaderboard (BFCL) V2 Live dataset as our base dataset. This dataset is particularly suitable for function-calling model training because:

1. Contains 2,251 question-function-answer pairs total
2. Provides diverse function calling scenarios:
   - 258 simple calls
   - 1,053 multiple parameter calls
   - 16 parallel function calls
   - 24 parallel multiple parameter calls
   - 882 irrelevance detection cases
   - 18 relevance detection cases
3. Offers complexity with an average of 3 function choices per entry (maximum 37)
4. Includes parameter diversity with an average of 4 parameters per function (maximum 28)

The dataset is processed and stored in optimized formats for efficient model training and evaluation.

In [ ]:
# upgrade boto3 
%pip install --upgrade pip --quiet
%pip install boto3 --upgrade --quiet
%pip install bcfl-eval

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

We need to set the project root so results are put in the correct location. 

In [12]:
import os
os.environ['BFCL_PROJECT_ROOT'] = os.getcwd()

If you're running this on your own machine, enter your AWS access keys in an .env file.
Uncomment the below cell to copy down an example .env.

In [ ]:
# # set up environment file
# %cp $(python -c "import bfcl_eval; print(bfcl_eval.__path__[0])")/.env.example $BFCL_PROJECT_ROOT/.env
# # Fill in necessary values in `.env`

In [ ]:
# download sample data
# %cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'test_case_ids_to_generate.json.example')") $BFCL_PROJECT_ROOT/test_case_ids_to_generate.json

For this example we're going to use a combination of v3_simple, v3_multiple, v3_live_relevance, and v3_irrelevance to train and evaluate with. For more information on these categories and their intents, please visit the [official BFCL documentation](https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard)

Let's move these to our local directory so we can begin preparing the data for distillation.

In [25]:
%mkdir questions
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'BFCL_v3_simple.json')") ./questions/BFCL_v3_simple.json
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'BFCL_v3_multiple.json')") ./questions/BFCL_v3_multiple.json
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'BFCL_v3_irrelevance.json')") ./questions/BFCL_v3_irrelevance.json
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'BFCL_v3_live_relevance.json')") ./questions/BFCL_v3_live_relevance..json

Now will grab the corresponding answers. For the simple and multiple datasets, we are provided possible answers and we'll use these for our mix-in labels.

Per the BFCL documentation, the correct answer for any question in the `BFCL_v3_irrelevance` datset is an empty list of functions, as these are design specifically to test the model's ability to correctly identify zero possible functions that are relevant.

The correct answer for any question in the `BFCL_v3_live_relevance` dataset is "at least one" function call returned.

Here's an excerpt from [their documentation](https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard):

> **Irrelevance Detection (875):** The scenario where none of the function choices provided are relevant to the user query and none should be invoked. We expect the model to not output a function call; the model can either output a message explaining why the function provided are not relevant or simply output a non-function call response (e.g., an empty list).

> **Relevance Detection (41):** The opposite of irrelevance detection. The scenario where at least one of the function choices provided are relevant to the user query and should be invoked, but the way the user prompt or the function doc is stated means that there could be infinitely many correct function calls and impossible to use a pre-defined possible answer set to evaluate. We expect the model to output some function call (one or multiple) that is relevant to the user query; we don't check for the correctness of the function call in this category (eg, correct parameter value).

In [26]:
%mkdir answers
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'possible_answer' / 'BFCL_v3_simple.json')") ./answers/BFCL_v3_simple.json
%cp $(python -c "import bfcl_eval, pathlib; print(pathlib.Path(bfcl_eval.__path__[0]) / 'data' / 'possible_answer' / 'BFCL_v3_multiple.json')") ./answers/BFCL_v3_multiple.json


To make things a bit cleaner, we'll create answer files for the relevance and irrelevance datasets as well. This will make it easier to combine for our training dataset as our mix-in will require a few ground truth examples.
We'll emulate the ground truth response structure the BFCL team is using, and return this for irrelevance answers:
```json
{"id": "irrelevance_13", "ground_truth": []}
```

and this for relevance answers, by picking a random function name from the list. Note that they "don't check for the correctness of the function call in this category (eg, correct parameter value)." so we'll use placeholders for the actual ground truth answers. Remember we're just doing this for labeled mix-in training data to hint our teacher model during distillation:
```json
{"id": "live_relevance_5-5-0", "ground_truth": [{"get_copyright_info": {"copyright_content": ["The specific content that is claimed to be copyrighted."], "copyright_holder": ["The name of the individual or organization that holds the copyright."], "confidence_score": [0.8]}}]}
```

## Data Preparation Steps

1. **Data Splitting**
   - Split the BFCL question datasets randomly:
     - 50% into `training` directory
     - 50% into `eval` directory
   
2. **Mix-in Data Creation**
   - From the training data:
     - Create `mix_in` subdirectory
     - Move 10% of records into mix_in
     - Keep 90% in training
   
3. **Ground Truth Integration**
   - For mix-in data:
     - Look up corresponding answers in answer dataset
     - Add as ground truth assistant responses
   
4. **Prompt Engineering**
   - Build Bedrock invoke API prompts with tool calling functionality
   
5. **Data Consolidation**
   - Combine all training data (including mix-in)
   - Format as JSONL for Bedrock distillation service
   - Save in training directory

In [ ]:
import json
import os
import pandas as pd
import numpy as np

# Create directories if they don't exist
os.makedirs('training', exist_ok=True)
os.makedirs('eval', exist_ok=True)

# Load and combine all question datasets
question_files = [
    'questions/BFCL_v3_simple.json',
    'questions/BFCL_v3_multiple.json',
    'questions/BFCL_v3_irrelevance.json',
    'questions/BFCL_v3_live_relevance.json'  # Note the double dot in filename
]

all_questions = []
for file in question_files:
    with open(file, 'r') as f:
        questions = json.load(f)
        all_questions.extend(questions)

# Convert to DataFrame for easier manipulation
df_questions = pd.DataFrame(all_questions)

# Randomly split into training (50%) and eval (50%)
df_train = df_questions.sample(frac=0.5, random_state=42)
df_eval = df_questions.drop(df_train.index)

# Save splits to respective directories
df_train.to_json('training/questions.json', orient='records', indent=2)
df_eval.to_json('eval/questions.json', orient='records', indent=2)

print(f"Training set size: {len(df_train)}")
print(f"Evaluation set size: {len(df_eval)}")

In [ ]:
# Create mix_in directory
os.makedirs('training/mix_in', exist_ok=True)

# Select 10% of training data for mix-in
df_mix_in = df_train.sample(frac=0.1, random_state=42)
df_train_remaining = df_train.drop(df_mix_in.index)

# Save mix-in and remaining training data
df_mix_in.to_json('training/mix_in/questions.json', orient='records', indent=2)
df_train_remaining.to_json('training/questions.json', orient='records', indent=2)

print(f"Mix-in set size: {len(df_mix_in)}")
print(f"Remaining training set size: {len(df_train_remaining)}")

In [ ]:
# Load answer datasets
answer_files = {
    'simple': 'answers/BFCL_v3_simple.json',
    'multiple': 'answers/BFCL_v3_multiple.json'
}

all_answers = {}
for dataset_type, file in answer_files.items():
    with open(file, 'r') as f:
        answers = json.load(f)
        all_answers.update({a['id']: a['ground_truth'] for a in answers})

# Add empty list answers for irrelevance cases
irrelevance_ids = df_mix_in[df_mix_in['id'].str.contains('irrelevance', na=False)]['id']
for id in irrelevance_ids:
    all_answers[id] = []

# Add placeholder answers for relevance cases
relevance_ids = df_mix_in[df_mix_in['id'].str.contains('live_relevance', na=False)]
for _, row in relevance_ids.iterrows():
    # Extract a random function from available choices
    if row['function_list']:
        func = np.random.choice(row['function_list'])
        all_answers[row['id']] = [{func: {"placeholder": "value"}}]

# Add ground truth answers to mix-in data
df_mix_in['ground_truth'] = df_mix_in['id'].map(all_answers)
df_mix_in.to_json('training/mix_in/questions_with_answers.json', orient='records', indent=2)

print(f"Added ground truth answers to {len(df_mix_in)} mix-in records")

In [ ]:
# System prompt for function calling
SYSTEM_PROMPT = """You are an AI assistant that helps users by calling appropriate functions based on their requests. When a function is relevant:
1. Select the most appropriate function from the available choices
2. Extract required parameters from the user's query
3. Return a JSON response with the function call details
4. If no functions are relevant, return an empty list []

Format your response as a JSON object with 'function_call' containing 'name' and 'parameters'."""

# Create JSONL records for all training data
def create_training_records():
    records = []
    
    # Process regular training data
    with open('training/questions.json', 'r') as f:
        train_data = json.load(f)
        for item in train_data:
            record = create_batch_inf_record(
                row={'id': item['id'], 'context': '', 'question': item['question']},
                system_prompt=SYSTEM_PROMPT,
                include_answer=False
            )
            records.append(record)
    
    # Process mix-in data with ground truth answers
    with open('training/mix_in/questions_with_answers.json', 'r') as f:
        mix_in_data = json.load(f)
        for item in mix_in_data:
            record = create_batch_inf_record(
                row={
                    'id': item['id'],
                    'context': '',
                    'question': item['question'],
                    'answers': {'text': json.dumps(item['ground_truth'])}
                },
                system_prompt=SYSTEM_PROMPT,
                include_answer=True
            )
            records.append(record)
    
    # Save combined training data as JSONL
    with open('training/training_data.jsonl', 'w') as f:
        for record in records:
            f.write(json.dumps(record) + '\n')
    
    print(f"Created {len(records)} training records in JSONL format")

# Generate the training data
create_training_records()

## Next Steps

Now that we have prepared our training data for the Bedrock distillation service, you can proceed to:

1. **Model Training**: Use the generated `training_data.jsonl` file to train your distilled model using the [Bedrock Model Distillation service](https://docs.aws.amazon.com/bedrock/latest/userguide/custom-models.html)

2. **Evaluation**: Use the data in the `eval` directory to assess your model's performance on:
   - Function selection accuracy
   - Parameter extraction quality
   - Handling of irrelevant queries
   - Response format consistency

3. **Fine-tuning**: Based on evaluation results, you may want to:
   - Adjust the mix-in percentage (currently 10%)
   - Modify the system prompt
   - Enhance the training data with additional examples

For more information on model distillation best practices, refer to the [Amazon Bedrock documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/custom-models-distillation.html).